In [41]:
# <Save_C3DFeatures_32Segments.m> -> python file


#-*- encoding: utf8 -*-
import os
import numpy as np
import struct

    ## <read_featureFC_file.py>(from 도롱빈)================================
def read_bin(input_file):
# input_file = open(r'.\normal_video_001\000032.fc6-1','rb')
    input_file = open(input_file,'rb')
    try:
        sizes = [struct.unpack('i',input_file.read(4))[0] for i in range(5)]
        m = np.prod(sizes)
        data = [struct.unpack('f',input_file.read(4))[0] for i in range(m)]
    finally:
        input_file
    feature_vector = np.array(data)

    return feature_vector, feature_vector.shape

    ## ======================================================================

C3D_Path = r'/content/drive/My Drive/warming-up project/c3d_ex/abnormal_video_fc'
C3D_Path_Seg = r'/content/drive/My Drive/warming-up project/c3d_ex/output_abnormal_final_c3d'


All_Folder = os.listdir(C3D_Path)
subcript = '_C.txt'

for ifolder in All_Folder:
    print(ifolder)
    # START 1 LOOP WITH 1 FC FOLDER(영상 1개)

    Folder_Path = str(C3D_Path) + "/" + str(ifolder)
    AllFiles = os.listdir(Folder_Path) # /.fc6-1 확장자 파일 싹다 리스트로 반환
                                       # fc6-1 files in feature directory, each file = a clip in video
                                       # one clip = 16 frames

    if len(AllFiles) == 0:
        print("no fc6-1 file in path")
        continue
    
    # .fc6-1 읽고 feature_vector 생성
    feature_vector = np.zeros((len(AllFiles), 4096))
    # each fc6-1 = 1 clips 16 frames = 4096-d ==> Total is [N/16]=88 clips like that
    # Iterate each fc6-1 file (16 frames each)
    for ifile in range(0,len(AllFiles)):
        FilePath =Folder_Path + '/' + AllFiles[ifile]

        data, s = read_bin(FilePath)
        feature_vector[ifile]=data # 1 row 4096-d in 88x4096 is assign by 1 clip feature (4096) ???????????? column을 row로 바꿈

    # 이제 fc6-1을 모두 읽어서 feature_vector(shape = (fc6-1 files 개수) x 4096)에 넣음


#----------------------------------------------------------------------------------
# 여기서부터 feature vector를 32seg로 나눔
# ((fc6-1 files 개수), 4096) -> (32, 4096)으로 만들어서 새로운 파일로 저장
# 여기서 만든 파일을 Training_AnomalyDetector_public.py에 넣어서 training 시킬 수 있음

    txt_file_name = C3D_Path_Seg + '/' + ifolder +subcript
    # feature txt name i.e abnormal_video_001_C.txt

    # fid1 = open(txt_file_name, 'w')

    sum_row = np.sum(feature_vector, axis=1) # 각 16 frame clip에 해당하는 row를 sum
    if np.where(sum_row==0)[0].size != 0:
        print('Error2')
    if np.isnan(feature_vector).sum() != 0:
        print('Error3')
    if np.inf in feature_vector:
        print('Error4')
 

    # 32 Segments
    Segments_Features = np.zeros((32, 4096))    # (32, 4096)
    thirty2_shots = np.linspace(0, len(AllFiles)-1, 33).astype(int) # AllFiles = .fc-1 확장자 파일들 ?????함수 안의 값 변경
    # SO: thirty2shots = [1 , 4, 6, 10, ..... 83, 85, 88], total elements is 33, vector 1x33
    
    count = -1
    # WRITE (88, 4096) TO (32, 4096)
    for ishots in range(0,len(thirty2_shots) - 1):  # ishorts starts from 0 to 31 (총 32개)
        ss = int(thirty2_shots[ishots])            # start clip index 
        ee = int(thirty2_shots[ishots + 1] - 1)     # end clip index 
        

        if ss == ee: # thirty2_shots에서 값의 차이가 1일 때
            temp_vect = feature_vector[ss] # ss==ee --> get 1 vector 4096-d from 88x4096

        elif ee < ss: # 영상 frame이 적어서 thirty2_shots에 같은 숫자가 나오는 경우
            temp_vect = feature_vector[ss]  # get 1 vector 4096-d from 88x4096

        else:
            temp_vect = feature_vector[ss:ee].mean(axis=0) # 각 열의 평균값을 가진 (1, 4096) 추출
                                                           # multiple rows vector -> one row vector

        # NORM-2 (L2)
        temp_vect = np.round(temp_vect / np.linalg.norm(temp_vect), 6)
        # divide by norm-2 (L2) of vector (Euclidean norm)=cumsum(sqrt(x[i]^2))

        if np.linalg.norm(temp_vect) == 0:
            print('Error5')

        count = count + 1 # next segment (max=32)
        Segments_Features[count]= temp_vect # push each segment feature to final 32 video segments feature



    # save 32 segment features in text file ( You can directly save and load .mat file in python as well).
    print(Segments_Features)
    # print(Segments_Features.shape)

    # for i in range(0,Segments_Features.shape[0]):
    #     feat_text = str(Segments_Features[i].tolist())
    #     fid1.write(feat_text)
    #     fid1.write('\n')
    np.savetxt(txt_file_name, Segments_Features, fmt="%.6f")

    # fid1.close()



abnormal_video_001
[[0.       0.       0.027733 ... 0.       0.       0.      ]
 [0.       0.       0.014779 ... 0.       0.       0.030944]
 [0.       0.       0.002643 ... 0.       0.       0.006323]
 ...
 [0.       0.       0.       ... 0.       0.       0.      ]
 [0.       0.       0.       ... 0.       0.       0.023944]
 [0.       0.       0.       ... 0.       0.       0.002591]]


In [34]:
import pandas as pd
ex1 = pd.read_csv('/content/drive/My Drive/warming-up project/c3d_ex/output_abnormal_final_c3d/abnormal_video_001_C.txt', delimiter=' ', header=None)
ex1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.0,0.0,0.027733,0.0,0.027292,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014678,0.0,0.005243,0.030184,0.010346,0.0,0.047060,0.0,0.044065,0.017433,0.060687,0.0,0.0,0.000000,0.000000,0.001436,0.059214,0.0,0.0,0.0,0.0,0.063253,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.023266,0.010330,0.027960,0.000000,0.0,0.000000,0.000562,0.0,0.0,0.005508,0.0,0.0,0.017861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.014779,0.0,0.029923,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005189,0.0,0.051400,0.011689,0.000000,0.0,0.016686,0.0,0.041286,0.051870,0.034811,0.0,0.0,0.000000,0.014163,0.000000,0.065848,0.0,0.0,0.0,0.0,0.046338,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.001282,0.007939,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002104,0.0,0.0,0.0,0.0,0.030944
2,0.0,0.0,0.002643,0.0,0.019655,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017110,0.0,0.043161,0.012741,0.000000,0.0,0.041888,0.0,0.012354,0.036720,0.034102,0.0,0.0,0.000000,0.000000,0.000000,0.068463,0.0,0.0,0.0,0.0,0.045786,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.038221,0.0,0.0,0.0,0.000000,0.0,0.000000,0.028777,0.005491,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.035140,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031265,0.0,0.0,0.0,0.0,0.006323
3,0.0,0.0,0.003924,0.0,0.051197,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000670,0.0,0.013821,0.002587,0.000000,0.0,0.050538,0.0,0.022456,0.006046,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.051513,0.0,0.0,0.0,0.0,0.028075,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.014231,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009619,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.0,0.035355,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015759,0.0,0.011537,0.016799,0.000000,0.0,0.026048,0.0,0.006988,0.021745,0.040148,0.0,0.0,0.009825,0.000000,0.000000,0.063478,0.0,0.0,0.0,0.0,0.030159,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.004008,0.0,0.0,0.0,0.000000,0.0,0.000000,0.008571,0.009549,0.000000,0.0,0.000000,0.012011,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.003340,0.0,0.049466,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002628,0.0,0.022322,0.000000,0.000000,0.0,0.026069,0.0,0.031165,0.018717,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.046889,0.0,0.0,0.0,0.0,0.059944,0.0,0.0,0.0,0.0,...,0.0,0.0,0.008575,0.0,0.0,0.006078,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.006940,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006969,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.0,0.003474,0.0,0.025266,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004572,0.0,0.029601,0.027757,0.000000,0.0,0.052153,0.0,0.031451,0.021124,0.074353,0.0,0.0,0.000000,0.002321,0.000000,0.063210,0.0,0.0,0.0,0.0,0.039595,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.018339,0.007622,0.015892,0.000000,0.0,0.000000,0.017100,0.0,0.0,0.000012,0.0,0.0,0.034888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009398,0.0,0.0,0.0,0.0,0.000000
7,0.0,0.0,0.000000,0.0,0.031401,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.017767,0.014500,0.000000,0.0,0.083591,0.0,0.000000,0.007206,0.044594,0.0,0.0,0.000000,0.017711,0.000000,0.060967,0.0,0.0,0.0,0.0,0.040032,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.014688,0.0,0.0,0.0,0.000000,0.0,0.002795,0.028303,0.021553,0.000000,0.0,0.000000,0.025751,0.0,0.0,0.030787,0.0,0.0,